In [1]:
# Importamos las clases que se requieren para manejar los agentes (Agent) y su entorno (Model).
# Cada modelo puede contener múltiples agentes.
from mesa import Agent, Model 

# Debido a que necesitamos que existe un solo agente por celda, elegimos ''SingleGrid''.
from mesa.space import MultiGrid

# Con ''RandomActivation'', hacemos que todos los agentes se activen ''al mismo tiempo''.
from mesa.time import RandomActivation

# Haremos uso de ''DataCollector'' para obtener información de cada paso de la simulación.
from mesa.datacollection import DataCollector

# matplotlib lo usaremos crear una animación de cada uno de los pasos del modelo.
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
plt.rcParams["animation.html"] = "jshtml"
matplotlib.rcParams['animation.embed_limit'] = 2**128

# Importamos los siguientes paquetes para el mejor manejo de valores numéricos.
import numpy as np
import pandas as pd

# Definimos otros paquetes que vamos a usar para medir el tiempo de ejecución de nuestro algoritmo.
import time
import datetime

#Importamos random para obtener coordenadas al azar
import random
import math

import ipywidgets as widgets
from IPython.display import display

#TYPES
#Type 1 Food
#Type 2 deposit 

In [2]:
class FoodAgent(Agent):
    def __init__(self, id, model):
        super().__init__(id, model)
        self.type
        self.has_food
        self.deposit_coordinates # Inicialmente no se conocen las coordenadas del depósito
        self.id
        self.food_coordinates   # Coordenadas para dirigirse a comida
        self.food.position      # Coordenadas de comida encontrada
        
#el método step del agente, se utiliza esta función para intentar obtener las coordenadas de la comida. 
#Si el agente no tiene comida, intentará moverse hacia la comida si hay coordenadas disponibles.

    def step(self):

        #Obtenemos las posibles celdas a las que el agente se puede mover
        neighborhood = self.model.grid.get_neighborhood(
            self.pos,
            moore=True,
            include_center=False)
        
        empty_cells = [cell for cell in neighborhood if (self.model.grid.is_cell_empty(cell))]                                                     
        
        food_cells = [cell for cell in empty_cells if (self.model.get_type(cell[0], cell[1]) > 0)] 
    
        deposit_cell = [cell for cell in empty_cells if (self.model.get_type(cell[0], cell[1]) == 0)] 
        
        self.model.update_food_coordinates(food_cells)

        self.model.update_deposit_coordinates(deposit_cell[0],deposit_cell[1])
 
        self.deposit_coordinates = self.model.get_deposit_coordinates()

        if empty_cells:

            if self.has_food == False:

                if self.model.get_type(self.pos[0],self.pos[1]) > 0:
                    self.model.take_food(self.pos[0],self.pos[1])
                    self.has_food = True
                    if self.deposit_coordinates:
                        self.move_to_food()
                    else:
                        empty_cell = self.random.choice(empty_cells)
                        self.model.grid.move_agent(self, empty_cell)
                    
                if food_cells:   
                        food_cell = self.random.choice(food_cells)
                        self.model.grid.move_agent(self, food_cell) 
                else:  
                    empty_cell = self.random.choice(empty_cells)
                    self.model.grid.move_agent(self, empty_cell)
                    
            else:
                # Si el agente tiene comida, muévelo hacia el depósito
                self.move_to_deposit()


    def update_food_coordinates(self):
        
        self.food_coordinates = self.model.update_food_coordinates(self.pos[0], self.pos[1])
        
    # Actualiza las coordenadas del depósito y notifica a otros agentes.
    def update_deposit_coordinates(self):
        xd = 1

    #get_food_coordinates en la clase Agent. Esta función verifica si hay coordenadas de comida en la lista self.food_coordinates. 
#Si hay coordenadas, toma las coordenadas de la primera comida en la lista, las elimina de la lista y las devuelve. Si no hay comida, devuelve None.

    def get_food_coordinates(self):
        """
        Recupera las coordenadas de la comida desde la lista self.food_coordinates.
        Devuelve las coordenadas de la primera comida encontrada y la elimina de la lista.
        Si no hay comida, devuelve None.
        """
        if self.food_coordinates:
            # Recupera las coordenadas de la primera comida en la lista
            food_position = self.food_coordinates.pop(0)
            return food_position
        else:
            return None
        
    #Mueve al agente hacia el depósito.
    def move_to_deposit(self):
        if self.deposit_coordinates is not None:
            deposit_row, deposit_col = self.deposit_coordinates
            current_row, current_col = self.position

            # Calcula la distancia euclidiana entre la posición actual y las coordenadas del depósito
            distance = math.sqrt((deposit_row - current_row)**2 + (deposit_col - current_col)**2)

            # Establece la velocidad del agente (puedes ajustar este valor según tus necesidades)
            speed = 1

            # Calcula las diferencias en las coordenadas para determinar la dirección del movimiento
            delta_row = (deposit_row - current_row) / distance * speed
            delta_col = (deposit_col - current_col) / distance * speed

            # Mueve al agente hacia las coordenadas del depósito
            self.position = (current_row + delta_row, current_col + delta_col)


    #Estas funciones suponen que has actualizado la clase Agent para incluir la información necesaria sobre la comida,
    #como food_position y has_food. Además, ten en cuenta que estas implementaciones asumen que los agentes pueden comunicarse 
    #directamente entre sí y que comparten información sobre la posición de la comida.
        
    def update_food(self, food_position):
        """
        Actualiza la información sobre la comida encontrada por el agente y notifica a otros agentes.
        """
        # Verifica si la comida está en la misma celda que el agente
        if self.pos == food_position:
            # Actualiza la información del agente sobre la comida
            self.has_food = True
            self.food_position = food_position

            # Notifica a otros agentes sobre la nueva comida
            for agent in self.model.schedule.agents:
                if agent != self:
                    agent.update_food_coordinates(food_position)
        else:
            # Si la comida no está en la misma celda, realiza otras acciones (ajustar esto según las necesidades)
            pass

    def move_to_food(self):
        """
        Mueve al agente hacia la posición de la comida.
        """
        if self.food_position is not None:
            food_row, food_col = self.food_position
            current_row, current_col = self.position

            # Calcula la distancia euclidiana entre la posición actual y las coordenadas de la comida
            distance = math.sqrt((food_row - current_row)**2 + (food_col - current_col)**2)

            # Establece la velocidad del agente (ajusta este valor según tus necesidades)
            speed = 1

            # Calcula las diferencias en las coordenadas para determinar la dirección del movimiento
            delta_row = (food_row - current_row) / distance * speed
            delta_col = (food_col - current_col) / distance * speed

            # Mueve al agente hacia las coordenadas de la comida
            self.position = (current_row + delta_row, current_col + delta_col)


In [3]:
#En get_grid obtenemos los contenidos de nuestra celda y seteamos valores numéricos
#para después setear los colores en el grid

def get_grid(model):
    grid = np.zeros((model.grid.width, model.grid.height), dtype=int)
    
    for (content, (x, y)) in model.grid.coord_iter():

        if content:
            grid[x][y] = 1   #Agent

        elif model.get_type(x, y) == 1: 
            grid[x][y] = 2  #Food

        elif model.get_type(x, y) == 2: 
            grid[x][y] = 3 #Deposit

        else:
            grid[x][y] = 0  # Empty cell

    return grid


In [4]:
class FoodModel(Model):
    def __init__ (self, width, height, num_agents, num_food):
        self.num_agents = num_agents
        self.grid = MultiGrid(width, height, torus = False)
        self.schedule = RandomActivation(self)
        self.datacollector = DataCollector(model_reporters={"Grid": get_grid})  
        self.type = np.zeros((width, height))
        self.deposit_coordinates
        self.food_coordinates = []
        self.is_deposit = False
        random.seed(12345)
        
        #añadimos
        #una id única a los agentes

        #self.type = 0 deposito
        #self.type = 1 comida

        id = 0

        #Place agents randomly on the grid
        for i in range(num_agents):
            x, y = random.randrange(self.grid.width), random.randrange(self.grid.height)

            while not (self.grid.is_cell_empty((x, y))):
                 x, y = random.randrange(self.grid.width), random.randrange(self.grid.height)    
            
            fooder = FoodAgent(id, self)
            self.grid.place_agent(fooder, (x, y))
            self.schedule.add(fooder)
            id = id + 1

        #Add random deposit coordinates
        x, y = random.randrange(self.grid.width), random.randrange(self.grid.height)
        if (self.grid.is_cell_empty((x, y))):
                self.type[x][y] = 0
                self.is_deposit[x][y] = 1
                self


             
    def step(self):
        self.step_call_count = 1

        if self.type[self.deposit_coordinates[0],self.deposit_coordinates[1]] == 47 :
            self.running = False

        else :
            if(self.step_call_count % 5 == 0):
                self.generate_food()    

        self.call_count += 1
        self.datacollector.collect(self)
        self.schedule.step() 


    def get_type(self, x,y):
        return self.type[x][y]
    
    def update_deposit(self):
        self.deposit += 1
        
    def generate_food(self, num_food):
        
        #Make food appear every 5 steps
        
        for i in range(self.food_quantity):      
            x, y = random.randrange(self.grid.width), random.randrange(self.grid.height)
            while not (self.grid.is_cell_empty(x, y) & self.get_type(x,y)== 0):
                 x, y = random.randrange(self.grid.width), random.randrange(self.grid.height) 

            self.type[x][y] = 1

            if num_food >= 5:
                choice = self.random.randint(1, 5)
            
            elif num_food == 4:
                choice = self.random.randint(1, 4)

            elif num_food == 3:
                choice = self.random.randint(1, 3)
            
            elif num_food == 2:
                choice = self.random.randint(1, 2)
            
            else:
                choice = 1

            num_food = max(0, num_food - choice)
            self.type[x][y] = choice

            if num_food <= 0:
                break

    def update_deposit_coordinates(self, x,y):
        self.deposit_coordinates = (x,y)

    def update_food_coordinates(self, cells):
        if cells:
            for cell in cells:
                x, y = cell
                if (x, y) not in self.food_coordinates:
                    self.food_coordinates.append((x, y))

    def take_food(self, x, y):
        self.type[x][y] -= 1

    def place_food(self, x, y):   
        self.type[x][y] += 1
        
    def get_food_coordinates(self, x, y):
        """
        Obtiene las coordenadas de la comida más cercana a la posición dada.
        """
        agent_position = (x, y)

        # Calcula las distancias entre la posición del agente y las coordenadas de la comida
        distances = [(math.sqrt((food_x - x)**2 + (food_y - y)**2), (food_x, food_y))
                     for (food_x, food_y) in self.food_coordinates]

        # Ordena las distancias de menor a mayor
        distances.sort()

        # Devuelve las coordenadas de la comida más cercana
        if distances:
            return distances[0][1]
        else:
            return None

    def get_deposit_coordinates(self, x, y):
        # Guarda las coordenadas en el arreglo food_coordinates
        return self.deposit_coordinates

In [ ]:
WIDTH = 20
HEIGHT = 20
NUM_AGENTS = 5
NUM_FOOD = 47
# Definimos el número máximo de steps a correr
MAX_STEPS = 1500

# Registramos el tiempo de inicio y ejecutamos la simulación
start_time = time.time()

model = FoodModel(WIDTH, HEIGHT, NUM_AGENTS, NUM_FOOD)

widgets.interact(model.step)

for i in range(MAX_STEPS):
    model.step()

# Imprimimos el tiempo que le tomó correr al modelo.

print('Tiempo de ejecución:', str(datetime.timedelta(seconds=(time.time() - start_time))))

UnboundLocalError: cannot access local variable 'Agent' where it is not associated with a value

In [ ]:
#Recolectamos los datos del grid 
all_grid = model.datacollector.get_model_vars_dataframe()

In [ ]:

fig, axs = plt.subplots(figsize=(7, 7))
axs.set_xticks([])
axs.set_yticks([])

# Definimos el color para la distinguir agentes, y celdas sucias
#el numero que seteamos anteriormente corresponde al valor del array
#de colores que queremos que se muestre 
cmap = plt.cm.colors.ListedColormap(['white', 'red', 'blue', 'green'])

patch = plt.imshow(all_grid.iloc[0][0], cmap=cmap, vmin=0, vmax=3)

def animate(i):
    patch.set_data(all_grid.iloc[i][0])
    patch.set_clim(vmin=0, vmax=2)  #establecemos los limites del color

anim = animation.FuncAnimation(fig, animate, frames=MAX_STEPS)